## Import libraries

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from tensorflow.keras.layers import Input, Dropout, Activation

from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import RobertaTokenizer, TFRobertaModel, RobertaConfig
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel, XLMRobertaConfig

## Load source datasets

In [2]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
train_df["excerpt_wordlen"] = train_df["excerpt"].apply(lambda x: len(str(x).split()))
train_df.drop(['url_legal','license','standard_error'], inplace=True, axis=1)
train_df.set_index("id", inplace=True)
print(f"train_df: {train_df.shape}\n")
train_df.head()

train_df: (2834, 3)



,excerpt,target,excerpt_wordlen
id,,,
c12129c31,When the young people returned to the ballroom...,-0.340259,179
85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,169
b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118,166
dd1000b26,And outside before the palace a great garden w...,-1.054013,164
37c1b32fb,Once upon a time there were Three Bears who li...,0.247197,147


In [3]:
test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
test_df["excerpt_wordlen"] = test_df["excerpt"].apply(lambda x: len(str(x).split()))
test_df.drop(['url_legal','license'], inplace=True, axis=1)
test_df.set_index("id", inplace=True)
print(f"test_df: {test_df.shape}\n")
test_df.head()

test_df: (7, 2)



,excerpt,excerpt_wordlen
id,,
c0f722661,My hope lay in Jack's promise that he would ke...,149
f0953f0a5,Dotty continued to go to Mrs. Gray's every nig...,181
0df072751,It was a bright and cheerful scene that greete...,174
04caf4e0c,Cell division is the process by which a parent...,180
0e63f8bea,Debugging is the process of finding and resolv...,168


## Extract target label

In [4]:
Ytrain = train_df['target'].values
Ytrain_strat = pd.qcut(train_df['target'].values, q=5, labels=range(0,5))
train_df.drop(['target'], inplace=True, axis=1)
print(f"Ytrain: {Ytrain.shape}")

Ytrain: (2834,)


## Model Hyperparameters

In [5]:
FOLD = 5
NUM_SEED = 1
VERBOSE = 1
MINI_BATCH_SIZE = 16
NUM_EPOCH = 20
MAX_LEN = max(train_df['excerpt_wordlen'].max(), 
              test_df['excerpt_wordlen'].max()) + 11

ROBERTA_BASE = "../input/huggingface-roberta-variants/roberta-base/roberta-base"
BERT_BASE_UNCASED = "../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased"
XLM_ROBERTA_BASE = "../input/huggingface-roberta-variants/tf-xlm-roberta-base/tf-xlm-roberta-base"
DISTILROBERTA_BASE = "../input/huggingface-roberta-variants/distilroberta-base/distilroberta-base"
DISTILBERT_BASE_UNCASED = "../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased"

## Helper Functions

In [6]:
def sent_encode(texts, tokenizer):
    input_ids = []
    attention_mask = []
    token_type_ids = []

    for text in tqdm(texts):
        tokens = tokenizer.encode_plus(text, max_length=MAX_LEN, truncation=True, 
                                       padding='max_length', add_special_tokens=True, 
                                       return_attention_mask=True, return_token_type_ids=True, 
                                       return_tensors='tf')
        
        input_ids.append(tokens['input_ids'])
        attention_mask.append(tokens['attention_mask'])
        token_type_ids.append(tokens['token_type_ids'])

    return np.array(input_ids), np.array(attention_mask), np.array(token_type_ids)

In [7]:
def rmse_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    return tf.math.sqrt(tf.math.reduce_mean((y_true - y_pred)**2))

In [8]:
def commonlit_model(transformer_model, use_tokens_type_ids=True):
    
    input_id = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
    token_type_id = Input(shape=(MAX_LEN,), dtype=tf.int32, name="token_type_ids")

    if use_tokens_type_ids:
        embed = transformer_model(input_id, token_type_ids=token_type_id, attention_mask=attention_mask)[0]
    
    else:
        embed = transformer_model(input_id, attention_mask=attention_mask)[0]
    
    embed = LayerNormalization()(embed)
    
    x = WeightNormalization(
            Conv1D(filters=384, kernel_size=5, 
                   strides=2, padding='same', 
                   kernel_regularizer=l2(0.0001),
                   kernel_initializer='he_uniform'))(embed)
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(rate=0.25)(x)
    
    x = WeightNormalization(
            Conv1D(filters=192, kernel_size=5, 
                   strides=2, padding='same', 
                   kernel_regularizer=l2(0.0001),
                   kernel_initializer='he_uniform'))(x)
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(rate=0.25)(x)
    
    x = Flatten()(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(units=1, kernel_initializer='lecun_normal')(x)

    model = Model(inputs=[input_id, attention_mask, token_type_id], outputs=x, 
                  name='CommonLit_Readability_Model')
    return model

## Roberta-Base Model

### Generate word tokens and attention masks

In [9]:
tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_BASE)

In [10]:
Xtrain_id, Xtrain_mask, Xtrain_token = sent_encode(train_df['excerpt'].values, tokenizer)

Xtrain_id = Xtrain_id.reshape((Xtrain_id.shape[0], Xtrain_id.shape[2]))
Xtrain_mask = Xtrain_mask.reshape((Xtrain_mask.shape[0], Xtrain_mask.shape[2]))
Xtrain_token = Xtrain_token.reshape((Xtrain_token.shape[0], Xtrain_token.shape[2]))
    
print(f"Input-ids: {Xtrain_id.shape} \nAttention Mask: {Xtrain_mask.shape} \nToken-type-ids: {Xtrain_token.shape}")

100%|██████████| 2834/2834 [00:06<00:00, 405.57it/s]


Input-ids: (2834, 216) 
Attention Mask: (2834, 216) 
Token-type-ids: (2834, 216)


In [11]:
Xtest_id, Xtest_mask, Xtest_token = sent_encode(test_df['excerpt'].values, tokenizer)

Xtest_id = Xtest_id.reshape((Xtest_id.shape[0], Xtest_id.shape[2]))
Xtest_mask = Xtest_mask.reshape((Xtest_mask.shape[0], Xtest_mask.shape[2]))
Xtest_token = Xtest_token.reshape((Xtest_token.shape[0], Xtest_token.shape[2]))
    
print(f"Input-ids: {Xtest_id.shape} \nAttention Mask: {Xtest_mask.shape} \nToken-type-ids: {Xtest_token.shape}")

100%|██████████| 7/7 [00:00<00:00, 523.00it/s]

Input-ids: (7, 216) 
Attention Mask: (7, 216) 
Token-type-ids: (7, 216)


### Initialize the Roberta-Base model

In [12]:
config = RobertaConfig.from_pretrained(ROBERTA_BASE)
config.output_hidden_states = False
transformer_model = TFRobertaModel.from_pretrained(ROBERTA_BASE, config=config)

Some layers from the model checkpoint at ../input/huggingface-roberta-variants/roberta-base/roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/huggingface-roberta-variants/roberta-base/roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [13]:
model = commonlit_model(transformer_model)
model.summary()

Model: "CommonLit_Readability_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 216)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 124645632   input_ids[0][0]                  
                                                                 attenti

### OOF Predictions

In [14]:
np.random.seed(23)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_final1 = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain_id, Ytrain_strat)):
        counter += 1

        train_x_id, train_x_mask, train_x_token = Xtrain_id[train], Xtrain_mask[train], Xtrain_token[train]
        val_x_id, val_x_mask, val_x_token = Xtrain_id[val], Xtrain_mask[val], Xtrain_token[val]
        train_y, val_y = Ytrain[train], Ytrain[val]

        model = commonlit_model(transformer_model)
        
        model.load_weights(f'../input/commonlit-roberta-variants-p1/Roberta-Base/CLRP_Roberta_Base_{counter}C.h5')
        
        y_pred = model.predict([val_x_id, val_x_mask, val_x_token])
        y_pred_final1 += model.predict([Xtest_id, Xtest_mask, Xtest_token])
        
        score = np.sqrt(mean_squared_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final1 = y_pred_final1 / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-83 | Fold-0 | OOF Score: 0.5818927901011692
Seed-83 | Fold-1 | OOF Score: 0.3159899519247878
Seed-83 | Fold-2 | OOF Score: 0.26385922701852865
Seed-83 | Fold-3 | OOF Score: 0.24831539858739213
Seed-83 | Fold-4 | OOF Score: 0.21069192041065657

Seed: 83 | Aggregate OOF Score: 0.32414985760850684


Aggregate OOF Score: 0.32414985760850684


## XLM-Roberta-Base Model

### Generate word tokens and attention masks

In [15]:
tokenizer = XLMRobertaTokenizer.from_pretrained(XLM_ROBERTA_BASE)

In [16]:
Xtrain_id, Xtrain_mask, Xtrain_token = sent_encode(train_df['excerpt'].values, tokenizer)

Xtrain_id = Xtrain_id.reshape((Xtrain_id.shape[0], Xtrain_id.shape[2]))
Xtrain_mask = Xtrain_mask.reshape((Xtrain_mask.shape[0], Xtrain_mask.shape[2]))
Xtrain_token = Xtrain_token.reshape((Xtrain_token.shape[0], Xtrain_token.shape[2]))
    
print(f"Input-ids: {Xtrain_id.shape} \nAttention Mask: {Xtrain_mask.shape} \nToken-type-ids: {Xtrain_token.shape}")

100%|██████████| 2834/2834 [00:02<00:00, 1021.71it/s]


Input-ids: (2834, 216) 
Attention Mask: (2834, 216) 
Token-type-ids: (2834, 216)


In [17]:
Xtest_id, Xtest_mask, Xtest_token = sent_encode(test_df['excerpt'].values, tokenizer)

Xtest_id = Xtest_id.reshape((Xtest_id.shape[0], Xtest_id.shape[2]))
Xtest_mask = Xtest_mask.reshape((Xtest_mask.shape[0], Xtest_mask.shape[2]))
Xtest_token = Xtest_token.reshape((Xtest_token.shape[0], Xtest_token.shape[2]))
    
print(f"Input-ids: {Xtest_id.shape} \nAttention Mask: {Xtest_mask.shape} \nToken-type-ids: {Xtest_token.shape}")

100%|██████████| 7/7 [00:00<00:00, 906.62it/s]

Input-ids: (7, 216) 
Attention Mask: (7, 216) 
Token-type-ids: (7, 216)


### Initialize the XLMRoberta-Base model

In [18]:
config = XLMRobertaConfig.from_pretrained(XLM_ROBERTA_BASE)
config.output_hidden_states = False
transformer_model = TFXLMRobertaModel.from_pretrained(XLM_ROBERTA_BASE, config=config)

Some layers from the model checkpoint at ../input/huggingface-roberta-variants/tf-xlm-roberta-base/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at ../input/huggingface-roberta-variants/tf-xlm-roberta-base/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [19]:
model = commonlit_model(transformer_model)
model.summary()

Model: "CommonLit_Readability_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 216)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
tfxlm_roberta_model (TFXLMRober TFBaseModelOutputWit 278043648   input_ids[0][0]                  
                                                                 attenti

### OOF Predictions

In [20]:
np.random.seed(29)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_final2 = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain_id, Ytrain_strat)):
        counter += 1

        train_x_id, train_x_mask, train_x_token = Xtrain_id[train], Xtrain_mask[train], Xtrain_token[train]
        val_x_id, val_x_mask, val_x_token = Xtrain_id[val], Xtrain_mask[val], Xtrain_token[val]
        train_y, val_y = Ytrain[train], Ytrain[val]

        model = commonlit_model(transformer_model)
        
        model.load_weights(f'../input/commonlit-roberta-variants-p1/XLM-Roberta-Base/CLRP_XLMRoberta_Base_{counter}C.h5')
        
        y_pred = model.predict([val_x_id, val_x_mask, val_x_token])
        y_pred_final2 += model.predict([Xtest_id, Xtest_mask, Xtest_token])
        
        score = np.sqrt(mean_squared_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final2 = y_pred_final2 / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-85 | Fold-0 | OOF Score: 0.6724309328381948
Seed-85 | Fold-1 | OOF Score: 0.434881209176005
Seed-85 | Fold-2 | OOF Score: 0.3692729401043699
Seed-85 | Fold-3 | OOF Score: 0.2977010268270374
Seed-85 | Fold-4 | OOF Score: 0.3885452823465845

Seed: 85 | Aggregate OOF Score: 0.43256627825843835


Aggregate OOF Score: 0.43256627825843835


## DistilRoberta-Base-Uncased Model

### Generate word tokens and attention masks

In [21]:
tokenizer = RobertaTokenizer.from_pretrained(DISTILROBERTA_BASE)

In [22]:
Xtrain_id, Xtrain_mask, Xtrain_token = sent_encode(train_df['excerpt'].values, tokenizer)

Xtrain_id = Xtrain_id.reshape((Xtrain_id.shape[0], Xtrain_id.shape[2]))
Xtrain_mask = Xtrain_mask.reshape((Xtrain_mask.shape[0], Xtrain_mask.shape[2]))
Xtrain_token = Xtrain_token.reshape((Xtrain_token.shape[0], Xtrain_token.shape[2]))
    
print(f"Input-ids: {Xtrain_id.shape} \nAttention Mask: {Xtrain_mask.shape} \nToken-type-ids: {Xtrain_token.shape}")

100%|██████████| 2834/2834 [00:05<00:00, 556.22it/s]


Input-ids: (2834, 216) 
Attention Mask: (2834, 216) 
Token-type-ids: (2834, 216)


In [23]:
Xtest_id, Xtest_mask, Xtest_token = sent_encode(test_df['excerpt'].values, tokenizer)

Xtest_id = Xtest_id.reshape((Xtest_id.shape[0], Xtest_id.shape[2]))
Xtest_mask = Xtest_mask.reshape((Xtest_mask.shape[0], Xtest_mask.shape[2]))
Xtest_token = Xtest_token.reshape((Xtest_token.shape[0], Xtest_token.shape[2]))
    
print(f"Input-ids: {Xtest_id.shape} \nAttention Mask: {Xtest_mask.shape} \nToken-type-ids: {Xtest_token.shape}")

100%|██████████| 7/7 [00:00<00:00, 537.87it/s]

Input-ids: (7, 216) 
Attention Mask: (7, 216) 
Token-type-ids: (7, 216)


### Initialize the DistilRoberta-Base model

In [24]:
config = RobertaConfig.from_pretrained(DISTILROBERTA_BASE)
config.output_hidden_states = False
transformer_model = TFRobertaModel.from_pretrained(DISTILROBERTA_BASE, config=config)

Some layers from the model checkpoint at ../input/huggingface-roberta-variants/distilroberta-base/distilroberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/huggingface-roberta-variants/distilroberta-base/distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [25]:
model = commonlit_model(transformer_model, use_tokens_type_ids=False)
model.summary()

Model: "CommonLit_Readability_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 216)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model_1 (TFRobertaMo TFBaseModelOutputWit 82118400    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
layer_normalization_36 (LayerNo (None, 216, 768)     1536        tf_robe

### OOF Predictions

In [26]:
np.random.seed(31)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_final3 = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain_id, Ytrain_strat)):
        counter += 1

        train_x_id, train_x_mask, train_x_token = Xtrain_id[train], Xtrain_mask[train], Xtrain_token[train]
        val_x_id, val_x_mask, val_x_token = Xtrain_id[val], Xtrain_mask[val], Xtrain_token[val]
        train_y, val_y = Ytrain[train], Ytrain[val]

        model = commonlit_model(transformer_model, use_tokens_type_ids=False)
        
        model.load_weights(f'../input/commonlit-roberta-variants-p1/DistilRoberta-Base/CLRP_DistilRoberta_Base_{counter}C.h5')
        
        y_pred = model.predict([val_x_id, val_x_mask, val_x_token])
        y_pred_final3 += model.predict([Xtest_id, Xtest_mask, Xtest_token])
        
        score = np.sqrt(mean_squared_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final3 = y_pred_final3 / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-82 | Fold-0 | OOF Score: 0.5335939518626852
Seed-82 | Fold-1 | OOF Score: 0.3364318069384966
Seed-82 | Fold-2 | OOF Score: 0.32787244656054687
Seed-82 | Fold-3 | OOF Score: 0.25105166712808297
Seed-82 | Fold-4 | OOF Score: 0.23226592424908787

Seed: 82 | Aggregate OOF Score: 0.3362431593477799


Aggregate OOF Score: 0.3362431593477799


## Bert-Base-Uncased Model

### Generate word tokens and attention masks

In [27]:
tokenizer = BertTokenizer.from_pretrained(BERT_BASE_UNCASED)

In [28]:
Xtrain_id, Xtrain_mask, Xtrain_token = sent_encode(train_df['excerpt'].values, tokenizer)

Xtrain_id = Xtrain_id.reshape((Xtrain_id.shape[0], Xtrain_id.shape[2]))
Xtrain_mask = Xtrain_mask.reshape((Xtrain_mask.shape[0], Xtrain_mask.shape[2]))
Xtrain_token = Xtrain_token.reshape((Xtrain_token.shape[0], Xtrain_token.shape[2]))
    
print(f"Input-ids: {Xtrain_id.shape} \nAttention Mask: {Xtrain_mask.shape} \nToken-type-ids: {Xtrain_token.shape}")

100%|██████████| 2834/2834 [00:15<00:00, 186.49it/s]


Input-ids: (2834, 216) 
Attention Mask: (2834, 216) 
Token-type-ids: (2834, 216)


In [29]:
Xtest_id, Xtest_mask, Xtest_token = sent_encode(test_df['excerpt'].values, tokenizer)

Xtest_id = Xtest_id.reshape((Xtest_id.shape[0], Xtest_id.shape[2]))
Xtest_mask = Xtest_mask.reshape((Xtest_mask.shape[0], Xtest_mask.shape[2]))
Xtest_token = Xtest_token.reshape((Xtest_token.shape[0], Xtest_token.shape[2]))
    
print(f"Input-ids: {Xtest_id.shape} \nAttention Mask: {Xtest_mask.shape} \nToken-type-ids: {Xtest_token.shape}")

100%|██████████| 7/7 [00:00<00:00, 201.74it/s]

Input-ids: (7, 216) 
Attention Mask: (7, 216) 
Token-type-ids: (7, 216)


### Initialize the Bert-Base model

In [30]:
config = BertConfig()
config.output_hidden_states = False
transformer_model = TFBertModel.from_pretrained(BERT_BASE_UNCASED, config=config)

Some layers from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [31]:
model = commonlit_model(transformer_model)
model.summary()

Model: "CommonLit_Readability_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 216)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attenti

### OOF Predictions

In [32]:
np.random.seed(23)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_final4 = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain_id, Ytrain_strat)):
        counter += 1

        train_x_id, train_x_mask, train_x_token = Xtrain_id[train], Xtrain_mask[train], Xtrain_token[train]
        val_x_id, val_x_mask, val_x_token = Xtrain_id[val], Xtrain_mask[val], Xtrain_token[val]
        train_y, val_y = Ytrain[train], Ytrain[val]

        model = commonlit_model(transformer_model)
        
        model.load_weights(f'../input/commonlit-bert-variants-p1/Bert-Base-Uncased/CLRP_Bert_Base_Uncased_{counter}C.h5')
        
        y_pred = model.predict([val_x_id, val_x_mask, val_x_token])
        y_pred_final4 += model.predict([Xtest_id, Xtest_mask, Xtest_token])
        
        score = np.sqrt(mean_squared_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final4 = y_pred_final4 / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-83 | Fold-0 | OOF Score: 0.5780486277112108
Seed-83 | Fold-1 | OOF Score: 0.29599539757095084
Seed-83 | Fold-2 | OOF Score: 0.34276718634690434
Seed-83 | Fold-3 | OOF Score: 0.3492470996393605
Seed-83 | Fold-4 | OOF Score: 0.2429676229549472

Seed: 83 | Aggregate OOF Score: 0.3618051868446747


Aggregate OOF Score: 0.3618051868446747


## DistilBert-Base-Uncased Model

### Generate word tokens and attention masks¶

In [33]:
tokenizer = DistilBertTokenizer.from_pretrained(DISTILBERT_BASE_UNCASED)

In [34]:
Xtrain_id, Xtrain_mask, Xtrain_token = sent_encode(train_df['excerpt'].values, tokenizer)

Xtrain_id = Xtrain_id.reshape((Xtrain_id.shape[0], Xtrain_id.shape[2]))
Xtrain_mask = Xtrain_mask.reshape((Xtrain_mask.shape[0], Xtrain_mask.shape[2]))
Xtrain_token = Xtrain_token.reshape((Xtrain_token.shape[0], Xtrain_token.shape[2]))
    
print(f"Input-ids: {Xtrain_id.shape} \nAttention Mask: {Xtrain_mask.shape} \nToken-type-ids: {Xtrain_token.shape}")

100%|██████████| 2834/2834 [00:15<00:00, 187.61it/s]


Input-ids: (2834, 216) 
Attention Mask: (2834, 216) 
Token-type-ids: (2834, 216)


In [35]:
Xtest_id, Xtest_mask, Xtest_token = sent_encode(test_df['excerpt'].values, tokenizer)

Xtest_id = Xtest_id.reshape((Xtest_id.shape[0], Xtest_id.shape[2]))
Xtest_mask = Xtest_mask.reshape((Xtest_mask.shape[0], Xtest_mask.shape[2]))
Xtest_token = Xtest_token.reshape((Xtest_token.shape[0], Xtest_token.shape[2]))
    
print(f"Input-ids: {Xtest_id.shape} \nAttention Mask: {Xtest_mask.shape} \nToken-type-ids: {Xtest_token.shape}")

100%|██████████| 7/7 [00:00<00:00, 114.64it/s]

Input-ids: (7, 216) 
Attention Mask: (7, 216) 
Token-type-ids: (7, 216)


### Initialize the DistilBert-Base model

In [36]:
config = DistilBertConfig()
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(DISTILBERT_BASE_UNCASED, config=config)

Some layers from the model checkpoint at ../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

In [37]:
model = commonlit_model(transformer_model, use_tokens_type_ids=False)
model.summary()

Model: "CommonLit_Readability_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 216)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 216)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
layer_normalization_72 (LayerNo (None, 216, 768)     1536        tf_dist

### OOF Predictions

In [38]:
np.random.seed(31)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_final5 = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain_id, Ytrain_strat)):
        counter += 1

        train_x_id, train_x_mask, train_x_token = Xtrain_id[train], Xtrain_mask[train], Xtrain_token[train]
        val_x_id, val_x_mask, val_x_token = Xtrain_id[val], Xtrain_mask[val], Xtrain_token[val]
        train_y, val_y = Ytrain[train], Ytrain[val]

        model = commonlit_model(transformer_model, use_tokens_type_ids=False)
        
        model.load_weights(f'../input/commonlit-bert-variants-p1/DistilBert-Base-Uncased/CLRP_DistilBert_Base_Uncased_{counter}C.h5')
        
        y_pred = model.predict([val_x_id, val_x_mask, val_x_token])
        y_pred_final5 += model.predict([Xtest_id, Xtest_mask, Xtest_token])
        
        score = np.sqrt(mean_squared_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final5 = y_pred_final5 / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-82 | Fold-0 | OOF Score: 0.5728470733668614
Seed-82 | Fold-1 | OOF Score: 0.3722774247360091
Seed-82 | Fold-2 | OOF Score: 0.19052898729738482
Seed-82 | Fold-3 | OOF Score: 0.15866087102496532
Seed-82 | Fold-4 | OOF Score: 0.13871715954737712

Seed: 82 | Aggregate OOF Score: 0.2866063031945195


Aggregate OOF Score: 0.2866063031945195


## Create submission file

In [39]:
y_pred_final = (y_pred_final1 * 0.15) + \
               (y_pred_final2 * 0.1) + \
               (y_pred_final3 * 0.15) + \
               (y_pred_final4 * 0.3) + \
               (y_pred_final5 * 0.3)

submit_df = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
submit_df['target'] = y_pred_final
submit_df.to_csv("./submission.csv", index=False)
submit_df.head()

,id,target
0,c0f722661,-0.581617
1,f0953f0a5,-0.563686
2,0df072751,-0.259267
3,04caf4e0c,-2.192328
4,0e63f8bea,-2.055590
